# **Citation Generation**

In [1]:
%pip install --quiet --upgrade bitsandbytes langchain langchain-community langchain-huggingface transformers beautifulsoup4 faiss-gpu rank_bm25 lark langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00


In [86]:
from langchain_core.documents import Document
from langchain.retrievers import EnsembleRetriever # Supports Ensembling of results from multiple retrievers
from langchain_community.retrievers import BM25Retriever
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_huggingface import ChatHuggingFace
from pydantic import BaseModel, Field
from typing import List
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata
from langchain import PromptTemplate
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
import os
import json
from google.colab import files
import time
from langchain_groq import ChatGroq
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from sentence_transformers import CrossEncoder

In [3]:
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

<br/>
<br/>
<br/>

## **Method: Direct Prompting with statement-wise citations**

Not all models support tool calling/function calling or have native JSON mode support. This method explores the use of direct prompting to ask the model to use a specific format

Additionally, we use few-shot prompting to enable in-context learning

### **Simple Experiment Data to test and observe behaviour**

In [5]:
docs = [
    Document(
        page_content="The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.",
        metadata={'country': 'Norway', 'source': 'visitNorway', 'link': 'https://www.visitnorway.com/'},
    ),
    Document(
        page_content="The most famous hikes in Norway include Preikestolen (a beautiful fjord), Kjeragbolten (with a famous boulder stuck between a mountain crevasse) as well as Trolltunga which resembes a tongue.",
        metadata={'country': 'Norway', 'source': 'norwayhikes', 'link': 'https://www.norwayhikes.com/'},
    ),
    Document(
        page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger.",
        metadata={'country': 'Iceland', 'source': 'IcelandTours', 'link': 'https://www.icelandtours.com/'},
    ),
    Document(
        page_content="Iceland is very famous for its fish freshly caught from the arctic sea. Famous dishes include the classic fish and chips, arctic cod and salmon soup!",
        metadata={'country': 'Iceland', 'source': 'IcelandGov', 'link': 'https://www.welcometoiceland.com/'},
    ),
    Document(
        page_content="The pasteries and bread in Iceland are fantastic, there are many bakeries in Iceland. One of the most popular bread is called dark rye bread ",
        metadata={'country': 'Iceland', 'source': 'IcelandFood', 'link': 'https://www.icelandicdelicacies.com/'},
    ),
    Document(
        page_content="Transportation within Reykjavik is fairly convenient as there is a public bus service called BSI. All you need to do is to download their mobile app, follow the instructions, and you're good to go. Transportation to places outside Reykjavik however requires a car. Some options include car rentals as well as booking bus tours.",
        metadata={'country': 'Iceland', 'source': 'IcelandBuses', 'link': 'https://www.icelandbuses.com/'},
    ),
    Document(
        page_content="Driving in Iceland is an amazing experience - open roads, majestic volcanos and towering mountains along the way, sheep and arctic foxes make it a great experience. All you need is an international driving license. And, please drive slowly during the winter!",
        metadata={'country': 'Iceland', 'source': 'IcelandBuses', 'link': 'https://www.icelandbuses.com/'},
    ),
    Document(
        page_content="Iceland is a must-go to place for adventurous people! You can hike active volcanoes, drive a jeep through the volcanic ash, explore a natural ice cave, see waterfalls. There are so many opportunities for an adventurer.",
        metadata={'country': 'Iceland', 'source': 'IcelandAdventures', 'link': 'https://www.icelandadventures.com/'},
    ),
    Document(
        page_content="One of the most famous diving sites in the world, Silfra, is located in Iceland! It is the only diving site in the world where you can dive between 2 tectonic plates. The water is also so fresh that you can drink from it, it is the best water that you will ever taste.",
        metadata={'country': 'Iceland', 'source': 'IcelandDiving', 'link': 'https://www.icelanddiving.com/'},
    ),
    Document(
        page_content="One of the most scenic hikes in Switzerland can be done at Grindelwald. At the summit of Grindelwald, a beautiful lake awaits you. However, you can only see this lake during summer time. Other notable hikes include Zermatt, i.e. the matterhorn and Lauterbrunnen.",
        metadata={'country': 'Switzerland', 'source': 'Swisstravels', 'link': 'https://www.switzerlandtravels.com/'},
    ),
    Document(
        page_content="The matterhorn at zermatt is a must-go for hiking enthusiasts. It is the icon of the famous chocolate: Toblerone. However, it is recommended to hire a mountain guide to go with you as it can be very dangerous!",
        metadata={'country': 'Switzerland', 'source': 'SwissHikes', 'link': 'https://www.switzerlandhiking.com/'},
    ),
]

### **Question**

In [6]:
question = "What can I eat in Iceland?"

In [7]:
prompt = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
For each statement in your answer, **you must include at least one inline numbered citation** (e.g., [1], [2]) for the Document Objects supporting it.
Always reset the numbering for citations to start from **1** for each response, and ensure the numbering increases sequentially.
Statements without sufficient support from the context should not be included in the answer.

At the bottom, provide the full citations corresponding to each number, but include **only the page content** of the Document Object (exclude metadata).

Write your response in a natural and coherent way, ensuring that the statements flow logically and transition smoothly between ideas. Use connecting words and phrases (e.g., "Additionally," "Furthermore," "For instance," "As a result,") to enhance readability.

### **IMPORTANT**:
1. Write your entire response as **a single paragraph**. Avoid using any new line characters in the response. All statements should flow naturally and seamlessly into one another.
2. If the answer cannot be found in the context, say "I don't know." **Do not include unsupported statements or make up information.**

Respond in the following format:
---
Statement 1 [1]. Statement 2 [2, 3].

Citations:
[1]: <Page Content 1> //Only the page content of the document
[2]: <Page Content 2>
[3]: <Page Content 3>
---

Here are a few examples:

---
The best hikes in Norway include the Reinebringen hike in the Lofoten islands, Preikestolen, Kjeragbolten, and Trolltunga [1, 2]. The Reinebringen hike, although not one of the highest peaks, offers an iconic view of the Reine fjord from its summit [1]. Preikestolen, Kjeragbolten, and Trolltunga are famous for their stunning fjord views and unique geological formations, such as a boulder stuck between a mountain crevasse and a tongue-shaped rock [2].

Citations:
[1]: "The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep."
[2]: "The most famous hikes in Norway include Preikestolen (a beautiful fjord), Kjeragbolten (with a famous boulder stuck between a mountain crevasse) as well as Trolltunga which resembles a tongue."
---

---
In Switzerland, you can embark on several scenic hikes [1]. One such hike is at Grindelwald, where at the summit, you will find a stunning lake, but it's only visible during the summer [1]. Other notable hikes include Zermatt, also known as the Matterhorn [1], and Lauterbrunnen [1]. For those seeking a challenging hike, the Matterhorn at Zermatt is a must-go [2]. This iconic peak is featured on the Toblerone chocolate and is best explored with a mountain guide due to the inherent dangers [2].

Citations:
[1]: "One of the most scenic hikes in Switzerland can be done at Grindelwald. At the summit of Grindelwald, a beautiful lake awaits you. However, you can only see this lake during summer time. Other notable hikes include Zermatt, i.e. the matterhorn and Lauterbrunnen."
[2]: "The matterhorn at zermatt is a must-go for hiking enthusiasts. It is the icon of the famous chocolate: Toblerone. However, it is recommended to hire a mountain guide to go with you as it can be very dangerous!"
---

---
In Iceland, you can participate in a variety of adventurous activities [1]. For instance, you can hike active volcanoes and explore a natural ice cave, offering unique geological experiences [1]. Driving in Iceland is also an amazing adventure, with open roads, majestic volcanoes, and towering mountains as your backdrop, and the possibility of encountering sheep and arctic foxes along the way [2]. Additionally, Iceland is known for its exceptional diving sites [3]. One of the most famous in the world, Silfra, is located in Iceland [3]. It is the only diving site where you can dive between two tectonic plates, and the water is so fresh that you can drink it, promising an unparalleled tasting experience [3].

Citations:
[1]: "Iceland is a must-go to place for adventurous people! You can hike active volcanoes, drive a jeep through the volcanic ash, explore a natural ice cave, see waterfalls. There are so many opportunities for an adventurer."
[2]: "Driving in Iceland is an amazing experience - open roads, majestic volcanos and towering mountains along the way, sheep and arctic foxes make it a great experience. All you need is an international driving license. And, please drive slowly during the winter!"
[3]: "One of the most famous diving sites in the world, Silfra, is located in Iceland! It is the only diving site in the world where you can dive between 2 tectonic plates. The water is also so fresh that you can drink from it, it is the best water that you will ever taste."
---

Question: {question}

Context: {context}

Helpful Answer:
"""


In [8]:
llm = ChatGroq()
llm_pipeline = llm | StrOutputParser()
response = llm_pipeline.invoke(prompt.format(question=question,context=docs))

In [9]:
response

'In Iceland, you can savor a variety of dishes that reflect its local culture and natural resources [3, 4]. The Baejarins Beztu Pylsur hot dog is a renowned street food, made of a mix of lamb, beef, and pork [3]. Iceland is also famous for its fresh fish, sourced directly from the Arctic sea, with popular dishes like the classic fish and chips, arctic cod, and salmon soup [4]. Additionally, Tommi\'s burger is another delicacy not to be missed [3]. Furthermore, Icelandic pastries and bread, such as the dark rye bread, are fantastic, and you can find many bakeries across the country [5].\n\nCitations:\n[3]: "The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi\'s burger."\n[4]: "Iceland is very famous for its fish freshly caught from the arctic sea. Famous dishes include the classic fish and chips, arctic cod and salmon soup!"\n[5]: "The

In [10]:
response.split('\n')

["In Iceland, you can savor a variety of dishes that reflect its local culture and natural resources [3, 4]. The Baejarins Beztu Pylsur hot dog is a renowned street food, made of a mix of lamb, beef, and pork [3]. Iceland is also famous for its fresh fish, sourced directly from the Arctic sea, with popular dishes like the classic fish and chips, arctic cod, and salmon soup [4]. Additionally, Tommi's burger is another delicacy not to be missed [3]. Furthermore, Icelandic pastries and bread, such as the dark rye bread, are fantastic, and you can find many bakeries across the country [5].",
 '',
 'Citations:',
 '[3]: "The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi\'s burger."',
 '[4]: "Iceland is very famous for its fish freshly caught from the arctic sea. Famous dishes include the classic fish and chips, arctic cod and salmon soup!

### **More Extensive Experiment Data**

In [11]:
data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

In [12]:
uploaded_files = files.upload()

Saving finland_articles_updated.csv to finland_articles_updated.csv


In [13]:
for file_name in uploaded_files.keys():
    os.rename(file_name, os.path.join(data_folder, file_name))

In [14]:
article_names = ['finland_articles_updated.csv']
article_fps = [os.path.join('.', 'data', article_name) for article_name in article_names]
docs = []
for article_fp in article_fps:
  df = pd.read_csv(article_fp)
  for _, row in df.iterrows():
    text = row['Title'] + " " + row['Content']

    doc = Document(
        page_content=text,
        metadata={'country': row['Country'], 'source': row['Source'], 'link': row['Article Links']}
    )

    docs.append(doc)

In [15]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250, chunk_overlap=50, add_start_index=True
)
chunked_docs = text_splitter.split_documents(docs)

In [16]:
num_docs_retrieved = 10
top_k = 10
bi_encoder_embeddings_model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings_model = HuggingFaceEmbeddings(model_name=bi_encoder_embeddings_model_name)
cross_encoder_embedings_model_name = "BAAI/bge-reranker-large"
cross_encoder_model = HuggingFaceCrossEncoder(model_name=cross_encoder_embedings_model_name)
compressor = CrossEncoderReranker(model=cross_encoder_model, top_n=top_k)
retriever_eval_res = {}
search_type = 'mmr'

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [17]:
vector_store_index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world")))
faiss_vector_store = FAISS(
  embedding_function=embeddings_model,
  index=vector_store_index,
  docstore=InMemoryDocstore(),
  index_to_docstore_id={})
faiss_vector_store.add_documents(chunked_docs)
faiss_retriever = faiss_vector_store.as_retriever(search_type=search_type, search_kwargs={"k": num_docs_retrieved})
reranked_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=faiss_retriever)

In [18]:
template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
For each statement in your answer, **you must include at least one inline numbered citation** (e.g., [1], [2]) for the Document Objects supporting it.
Always reset the numbering for citations to start from **1** for each response, and ensure the numbering increases sequentially.
Statements without sufficient support from the context should not be included in the answer.

At the bottom, provide the full citations corresponding to each number, but include **only the page content** of the Document Object (exclude metadata).

Write your response in a natural and coherent way, ensuring that the statements flow logically and transition smoothly between ideas. Use connecting words and phrases (e.g., "Additionally," "Furthermore," "For instance," "As a result,") to enhance readability.

### **IMPORTANT**:
1. Write your entire response as **a single paragraph**. Avoid using any new line characters in the response. All statements should flow naturally and seamlessly into one another.
2. If the answer cannot be found in the context, say "I don't know." **Do not include unsupported statements or make up information.**

Respond in the following format:
---
Statement 1 [1]. Statement 2 [2, 3].

Citations:
[1]: <Page Content 1> //Only the page content of the document
[2]: <Page Content 2>
[3]: <Page Content 3>
---

Here are a few examples:

---
The best hikes in Norway include the Reinebringen hike in the Lofoten islands, Preikestolen, Kjeragbolten, and Trolltunga [1,2]. The Reinebringen hike, although not one of the highest peaks, offers an iconic view of the Reine fjord from its summit [1]. Preikestolen, Kjeragbolten, and Trolltunga are famous for their stunning fjord views and unique geological formations, such as a boulder stuck between a mountain crevasse and a tongue-shaped rock [2].

Citations:
[1]: "The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep."
[2]: "The most famous hikes in Norway include Preikestolen (a beautiful fjord), Kjeragbolten (with a famous boulder stuck between a mountain crevasse) as well as Trolltunga which resembles a tongue."
---

---
In Switzerland, you can embark on several scenic hikes [1]. One such hike is at Grindelwald, where at the summit, you will find a stunning lake, but it's only visible during the summer [1]. Other notable hikes include Zermatt, also known as the Matterhorn [1], and Lauterbrunnen [1]. For those seeking a challenging hike, the Matterhorn at Zermatt is a must-go [2]. This iconic peak is featured on the Toblerone chocolate and is best explored with a mountain guide due to the inherent dangers [2].

Citations:
[1]: "One of the most scenic hikes in Switzerland can be done at Grindelwald. At the summit of Grindelwald, a beautiful lake awaits you. However, you can only see this lake during summer time. Other notable hikes include Zermatt, i.e. the matterhorn and Lauterbrunnen."
[2]: "The matterhorn at zermatt is a must-go for hiking enthusiasts. It is the icon of the famous chocolate: Toblerone. However, it is recommended to hire a mountain guide to go with you as it can be very dangerous!"
---

---
In Iceland, you can participate in a variety of adventurous activities [1]. For instance, you can hike active volcanoes and explore a natural ice cave, offering unique geological experiences [1]. Driving in Iceland is also an amazing adventure, with open roads, majestic volcanoes, and towering mountains as your backdrop, and the possibility of encountering sheep and arctic foxes along the way [2]. Additionally, Iceland is known for its exceptional diving sites [3]. One of the most famous in the world, Silfra, is located in Iceland [3]. It is the only diving site where you can dive between two tectonic plates, and the water is so fresh that you can drink it, promising an unparalleled tasting experience [3].

Citations:
[1]: "Iceland is a must-go to place for adventurous people! You can hike active volcanoes, drive a jeep through the volcanic ash, explore a natural ice cave, see waterfalls. There are so many opportunities for an adventurer."
[2]: "Driving in Iceland is an amazing experience - open roads, majestic volcanos and towering mountains along the way, sheep and arctic foxes make it a great experience. All you need is an international driving license. And, please drive slowly during the winter!"
[3]: "One of the most famous diving sites in the world, Silfra, is located in Iceland! It is the only diving site in the world where you can dive between 2 tectonic plates. The water is also so fresh that you can drink from it, it is the best water that you will ever taste."
---

Question: {question}

Context: {context}

Helpful Answer:
"""

prompt_template = PromptTemplate.from_template(template)

In [20]:
llm = ChatGroq()
llm_pipeline = llm | StrOutputParser()
pipeline = ( prompt_template | llm | StrOutputParser()  )

### **Using Question**

In [21]:
question = "What hikes can I do in Finland?"

In [24]:
retrieved_docs = reranked_retriever.invoke(question)

response = pipeline.invoke({"question": question,"context": retrieved_docs})

## **Evaluate Citations**

In [63]:
answer_split = response.split('\n')

In [64]:
answer_split

["In Finland, you can find several popular hiking trails, including Karhunkierros in Kuusamo, Hetta-Pallas Trail in Western Lapland, and Pyhä-Luosto Trail in Central Lapland [1]. Additionally, there is the famous Karhunkierros trail located in the Kuusamo region, and if you're not up for an 80-kilometer trek, you can opt for shorter hikes in Oulanka, Riisitunturi, or Hossa National Park [2]. Before your hike, it's essential to consider your experience level and the trail's rating, as weather conditions can be a significant factor, especially in winter [3].",
 '',
 'Citations:',
 '[1]: "The most popular trails in Finland include Karhunkierros in Kuusamo, Hetta-Pallas Trail in Western Lapland, and Pyhä-Luosto Trail in Central Lapland. If you\'re looking for"',
 '[2]: "One of the most famous hiking trails in Finland, Karhunkierros, is located here, but if trekking 80-kilometres isn’t your idea of a good time, you can take a shorter hike in Oulanka, Riisitunturi, or Hossa National Park – j

In [65]:
answer = answer_split[0]
citations = answer_split[3:]

citations_dict = {}
for citation in citations:
    match = re.match(r"\[(.*?)\]", citation)
    if match:
        citation_number = match.group(1)
        citation_content = citation.split(':')[1].strip()
        citations_dict[citation_number] = citation_content

In [66]:
citations_dict

{'1': '"The most popular trails in Finland include Karhunkierros in Kuusamo, Hetta-Pallas Trail in Western Lapland, and Pyhä-Luosto Trail in Central Lapland. If you\'re looking for"',
 '2': '"One of the most famous hiking trails in Finland, Karhunkierros, is located here, but if trekking 80-kilometres isn’t your idea of a good time, you can take a shorter hike in Oulanka, Riisitunturi, or Hossa National Park – just be"',
 '3': '"weather conditions, however, are something to do consider always – especially in the winter. Trails in Finnish national parks are rated from easy to demanding. Before your hike, check the rating of your chosen trail. Coloured signposts on marked routes will"'}

In [67]:
statements = [s.strip() for s in answer.split('.') if s.strip()]

In [68]:
statements

['In Finland, you can find several popular hiking trails, including Karhunkierros in Kuusamo, Hetta-Pallas Trail in Western Lapland, and Pyhä-Luosto Trail in Central Lapland [1]',
 "Additionally, there is the famous Karhunkierros trail located in the Kuusamo region, and if you're not up for an 80-kilometer trek, you can opt for shorter hikes in Oulanka, Riisitunturi, or Hossa National Park [2]",
 "Before your hike, it's essential to consider your experience level and the trail's rating, as weather conditions can be a significant factor, especially in winter [3]"]

In [69]:
statement_citations = {}
for statement in statements:
    inline_citations = re.findall(r"\[(.*?)\]", statement)
    for citation in inline_citations:
        if citation in citations_dict:
            statement = statement.replace(f"[{citation}]", "").strip()
    statement_citations[statement] = [citations_dict[citation] for citation in inline_citations if citation in citations_dict]

In [70]:
statement_citations

{'In Finland, you can find several popular hiking trails, including Karhunkierros in Kuusamo, Hetta-Pallas Trail in Western Lapland, and Pyhä-Luosto Trail in Central Lapland': ['"The most popular trails in Finland include Karhunkierros in Kuusamo, Hetta-Pallas Trail in Western Lapland, and Pyhä-Luosto Trail in Central Lapland. If you\'re looking for"'],
 "Additionally, there is the famous Karhunkierros trail located in the Kuusamo region, and if you're not up for an 80-kilometer trek, you can opt for shorter hikes in Oulanka, Riisitunturi, or Hossa National Park": ['"One of the most famous hiking trails in Finland, Karhunkierros, is located here, but if trekking 80-kilometres isn’t your idea of a good time, you can take a shorter hike in Oulanka, Riisitunturi, or Hossa National Park – just be"'],
 "Before your hike, it's essential to consider your experience level and the trail's rating, as weather conditions can be a significant factor, especially in winter": ['"weather conditions, ho

**Citation Recall**

In [71]:
def process_response(response):
  answer_split = response.split('\n')

  answer = answer_split[0]
  citations = answer_split[3:]

  citations_dict = {}
  for citation in citations:
      match = re.match(r"\[(.*?)\]", citation)
      if match:
          citation_number = match.group(1)
          citation_content = citation.split(':')[1].strip()
          citations_dict[citation_number] = citation_content

  statements = [s.strip() for s in answer.split('.') if s.strip()]

  statement_citations = {}
  for statement in statements:
      inline_citations = re.findall(r"\[(.*?)\]", statement)
      for citation in inline_citations:
          if citation in citations_dict:
              statement = statement.replace(f"[{citation}]", "").strip()
      statement_citations[statement] = [citations_dict[citation] for citation in inline_citations if citation in citations_dict]

  return statement_citations

In [72]:
process_response(response)

{'In Finland, you can find several popular hiking trails, including Karhunkierros in Kuusamo, Hetta-Pallas Trail in Western Lapland, and Pyhä-Luosto Trail in Central Lapland': ['"The most popular trails in Finland include Karhunkierros in Kuusamo, Hetta-Pallas Trail in Western Lapland, and Pyhä-Luosto Trail in Central Lapland. If you\'re looking for"'],
 "Additionally, there is the famous Karhunkierros trail located in the Kuusamo region, and if you're not up for an 80-kilometer trek, you can opt for shorter hikes in Oulanka, Riisitunturi, or Hossa National Park": ['"One of the most famous hiking trails in Finland, Karhunkierros, is located here, but if trekking 80-kilometres isn’t your idea of a good time, you can take a shorter hike in Oulanka, Riisitunturi, or Hossa National Park – just be"'],
 "Before your hike, it's essential to consider your experience level and the trail's rating, as weather conditions can be a significant factor, especially in winter": ['"weather conditions, ho

In [73]:
# https://www.sbert.net/docs/cross_encoder/pretrained_models.html#nli
nli_model = CrossEncoder("cross-encoder/nli-deberta-v3-base")

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/738M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/417 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
def citation_recall_response(response):
  statement_citations = process_response(response)
  entailment_count = 0
  for k,v in statement_citations:
    logits = nli_model.predict(k,'.'.join(v))
    if logits.argmax()==1:
      entailment_count+=1
  return entailment_count/len(statement_citations)

<br/>
<br/>
<br/>
<br/>
<br/>

## **Conclusions**

We decide to use LLMs from LLM providers such as <u>ChatGroq</u> due to the fast inference speed and ability to output well-structured outputs which makes it easy for formatting